#Prova
Resumo para a prova do dia 26/04/2023

Conteúdo da prova
- Classificadores
- Regressores

Resumo:
Em Machine Learning, existem dois tipos principais de problemas: classificação e regressão. Os algoritmos de aprendizado de máquina podem ser classificados em classificadores e regressores de acordo com a natureza do problema que estão resolvendo.

Os classificadores são usados para problemas de classificação, onde a tarefa é atribuir uma classe a um conjunto de dados. Em outras palavras, o objetivo do classificador é prever a categoria de um determinado conjunto de dados. Exemplos de problemas de classificação incluem: prever se um e-mail é spam ou não, prever se um paciente tem uma doença específica com base em seus sintomas, classificar imagens em diferentes categorias, entre outros. Alguns exemplos de algoritmos de classificação são: Naive Bayes, Árvore de Decisão, Random Forest, K-Nearest Neighbors (KNN), Support Vector Machines (SVM), entre outros.

Os regressores, por outro lado, são usados para problemas de regressão, onde a tarefa é prever um valor numérico para um conjunto de dados. Em outras palavras, o objetivo do regressor é prever um valor contínuo. Exemplos de problemas de regressão incluem: prever a demanda futura de um produto, prever a temperatura, prever o preço de uma casa com base em suas características, entre outros. Alguns exemplos de algoritmos de regressão são: Regressão Linear, Regressão Logística, Árvores de Regressão, KNN Regressor, SVM Regressor, entre outros.

Em resumo, a principal diferença entre classificadores e regressores é o tipo de saída que eles produzem. Enquanto os classificadores produzem uma classe ou categoria, os regressores produzem um valor numérico.

In [12]:
# Faz os imports necessários
# Lembrar de colocar aqui a base de dados
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.datasets import load_digits, load_diabetes
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from matplotlib.font_manager import dataclasses
import numpy as np
import pandas as pd
import os
import random

# Valor da seed a ser utulizada
RANDOM_SEED = 42

# PYTHONHASHSEED environment variable
os.environ['PYTHONHASHSEED']=str(RANDOM_SEED)

# python built-in pseudo-random generator
random.seed(RANDOM_SEED)

# numpy
np.random.seed(RANDOM_SEED)


# Carregando os dados
data = pd.read_csv("spambase.csv", header=None)
# X,y = load_digits(return_X_y=True)

# Transformando em Numpy
val=data.values
att=data.columns
X=val[:,0:57]
y=val[:,57]
# Mostrando o formato de X e y
print(X.shape)
print(y.shape)

(4601, 57)
(4601,)


##Classificadores
Um classificador é um modelo de machine learning que é usado para classificar dados em classes predefinidas. Essas classes podem ser binárias (duas classes) ou multiclasse (mais de duas classes). Por exemplo, um classificador pode ser usado para classificar imagens de frutas em maçãs e bananas.

###KNN
___
O classificador K-Nearest Neighbors (KNN) é um dos algoritmos mais simples de aprendizado supervisionado em machine learning. A ideia principal do KNN é classificar um ponto de dados baseado em seus vizinhos mais próximos na base de treinamento. Esse algoritmo considera os K vizinhos mais próximos do ponto de teste e atribui a classe mais frequente entre esses vizinhos ao ponto de teste.

O valor de K é escolhido pelo usuário e afeta o resultado da classificação. Se K é escolhido muito pequeno, o modelo pode ser afetado por ruído e outliers, enquanto se K é escolhido muito grande, a região de decisão pode se tornar muito genérica e não ajustada aos dados. Portanto, a escolha do valor de K é um hiper

In [3]:
# Exemplo KNN
X, X_val, y, y_val=train_test_split(X,y, train_size=0.8, random_state=42, stratify=y)

Knn = KNeighborsClassifier()
parameters = [{
                'n_neighbors': [1, 3, 5, 7, 9, 11], 
                'weights': ['uniform', 'distance'],
                'p':[1,2],
                'metric': ['euclidean', 'manhattan']
              }]

# Executando o GridSearchCV
gs = GridSearchCV(Knn, parameters, scoring = 'accuracy', cv=5, n_jobs=-1)
gs.fit(X_val, y_val)

# Mostrando os melhores parâmetros
print("Melhores parâmetros encontrados: ", gs.best_params_)

Knn = gs.best_estimator_
score = model_selection.cross_val_score(Knn, X, y, cv=5)
print("Acurácia média: ", score.mean())
print("Desvio padrão: ", score.std())
print("Acurácia por fold: ", score)

#Gerando matriz de confusão com o resultado da validação cruzada
ypred=model_selection.cross_val_predict(Knn, X, y, cv=5)

# Precisão
prec=precision_score(y, ypred, average='weighted')
print("Precision:", prec)

# Revocação (Recall)
recall=recall_score(y, ypred, average='weighted')
print("Revocação:", recall)

# f1 score
f1 = f1_score(y, ypred, average='macro')
print("f1 score:", f1)

# Matriz de confusão
cm=confusion_matrix(y, ypred)
print(cm)

Melhores parâmetros encontrados:  {'metric': 'manhattan', 'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Acurácia média:  0.8595108695652174
Desvio padrão:  0.015168788844513965
Acurácia por fold:  [0.86413043 0.85733696 0.875      0.83152174 0.86956522]
Precision: 0.859110069629792
Revocação: 0.8595108695652174
f1 score: 0.8523764775368312
[[1986  244]
 [ 273 1177]]


###Árvore de Decisão
___
Árvores de Decisão são algoritmos de aprendizagem de máquina supervisionados utilizados para resolver problemas de classificação e regressão. Esses algoritmos constroem uma estrutura em forma de árvore, onde cada nó interno representa uma decisão a ser tomada e cada folha representa uma classe ou valor predito. A construção da árvore é feita a partir da análise dos atributos e seus valores nos dados de treinamento, onde cada divisão na árvore é feita com base no atributo que fornece a maior informação.

Existem diversos algoritmos para construir árvores de decisão, sendo o mais comum o ID3 (Iterative Dichotomiser 3), que usa o ganho de informação como critério de seleção de atributos, e o C4.5, uma versão melhorada do ID3 que utiliza a razão de ganho de informação como critério. Outros algoritmos conhecidos são o CART (Classification and Regression Trees), o CHAID (Chi-square Automatic Interaction Detector) e o MARS (Multivariate Adaptive Regression Splines).

As árvores de decisão são fáceis de interpretar e de explicar, o que as torna populares em diversas áreas de aplicação. Além disso, elas lidam bem com dados categóricos e numéricos, e são capazes de lidar com dados ausentes. No entanto, podem ser sensíveis a ruídos e dados irrelevantes, e podem criar árvores muito complexas que levam ao overfitting dos dados de treinamento.

Para avaliar a qualidade da árvore de decisão construída, são utilizadas métricas como a acurácia, a precisão, a revocação e a F1-Score. Além disso, pode ser realizada a poda da árvore para evitar o overfitting e melhorar o desempenho na fase de teste.

In [ ]:
# Exemplo de Arvore de Decisão

# Exemplo de como Normalização dos dados
# scaler=preprocessing.MinMaxScaler().fit(X)
# X=scaler.transform(X)
X, X_val, y, y_val=train_test_split(X,y, train_size=0.8, random_state=42, stratify=y)

model = DecisionTreeClassifier()

parameters = [{'criterion': ['gini', 'entropy', 'log_loss'], 'splitter': ['best', 'random']},]

# GridSearch para customizar os parâmetros sobre base de validação
gs = GridSearchCV(model, parameters, scoring = 'accuracy', cv=5, n_jobs=-1)
gs.fit(X_val, y_val)

print("Melhores parâmetros encontrados: ", gs.best_params_)

# Definindo a técnica a ser utilizada
model = gs.best_estimator_

# Validação cruzada com 5 folds
score=model_selection.cross_val_score(model,X,y,cv=5)
print("Acurácia média:", score.mean())
print("Desvio padrão:", score.std())

# Fornece a saída para cada exemplo de teste
ypred=model_selection.cross_val_predict(model,X,y,cv=5)

# Precisão
prec=precision_score(y, ypred, average='weighted')
print("Precision:", prec)

# Revocação (Recall)
recall=recall_score(y, ypred, average='weighted')
print("Revocação:", recall)

# f1 score
f1 = f1_score(y, ypred, average='macro')
print("f1 score:", f1)

# Matriz de confusão
cm=confusion_matrix(y, ypred)
print(cm)

Melhores parâmetros encontrados:  {'criterion': 'log_loss', 'splitter': 'random'}
Acurácia média: 0.9031882702147073
Desvio padrão: 0.016769148961738995
Precision: 0.9022359433048447
Revocação: 0.9021739130434783
f1 score: 0.8976338253446687
[[1638  146]
 [ 142 1018]]


###SVM
___
SVM (Support Vector Machine) é um algoritmo de aprendizado de máquina que se baseia em encontrar um hiperplano que separe duas classes de amostras. O SVM busca o hiperplano que maximize a distância entre as amostras mais próximas de cada classe, chamadas de vetores de suporte. Quando as amostras não são linearmente separáveis, o SVM utiliza uma técnica chamada kernel trick para mapeá-las em um espaço de maior dimensão em que elas se tornam linearmente separáveis. O SVM é comumente utilizado em problemas de classificação, mas também pode ser utilizado em problemas de regressão.

In [ ]:
# Exemplo SVM
X, X_val, y, y_val=train_test_split(X,y, train_size=0.8, random_state=42, stratify=y)

classifier = SVC(kernel = 'rbf')

# Parâmetros a serem usados no pelo GridSearchCV
parameters = [{'C' : [1,10,1000],
              'gamma': [0.001, 0.0001],
              'kernel' : ['rbf'],
              'random_state' : [42]}]

# Executando o GridSearchCV
gs=GridSearchCV(classifier, parameters, cv=5, n_jobs=-1)
gs.fit(X_val, y_val)

print("Melhores parâmetros encontrados: ", gs.best_params_)

classifier = gs.best_estimator_
score = model_selection.cross_val_score(classifier, X, y, cv=5)
print("Acurácia média: ", score.mean())
print("Desvio padrão: ", score.std())
print("Acurácia por fold: ", score)

ypred=model_selection.cross_val_predict(classifier,X,y,cv=5)

# Precisão
prec=precision_score(y, ypred, average='weighted')
print("Precision:", prec)

# Revocação (Recall)
recall=recall_score(y, ypred, average='weighted')
print("Revocação:", recall)

# f1 score
f1=f1_score(y, ypred, average='macro')
print("f1 score:", f1)

# Matriz de confusão
cm=confusion_matrix(y, ypred)
print(cm)

Melhores parâmetros encontrados:  {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf', 'random_state': 42}
Acurácia média:  0.8955414012738855
Desvio padrão:  0.00433037750623591
Acurácia por fold:  [0.89596603 0.90021231 0.88747346 0.89808917 0.89596603]
Precision: 0.8952484230432137
Revocação: 0.8955414012738854
f1 score: 0.8896761411158186
[[1326  101]
 [ 145  783]]


###Naive Bayes
___
Classificador probabilístico que utiliza o Teorema de Bayes para determinar a probabilidade de uma amostra pertencer a uma determinada classe. Ele assume que cada atributo é independente dos outros atributos e que a presença de um determinado atributo em uma classe não afeta a probabilidade de outro atributo estar presente nessa mesma classe. O Naive Bayes é bastante utilizado em problemas de classificação de texto e análise de sentimento, por exemplo.

In [4]:
# Exemplo de Naive Bayes

X, X_val, y, y_val=train_test_split(X,y, train_size=0.8, stratify=y)

gnb = GaussianNB()

parameters = [{'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05],
               'priors': [None, [0.2, 0.8], [0.3, 0.7], [0.4, 0.6], [0.1, 0.9]]
              }]

gs=GridSearchCV(gnb, parameters, cv=5, n_jobs=-1)
gs.fit(X_val, y_val)

print("Melhores parâmetros encontrados: ", gs.best_params_)

gnb = gs.best_estimator_
score = model_selection.cross_val_score(gnb, X, y, cv=5)
print("Acurácia média: ", score.mean())
print("Desvio padrão: ", score.std())
print("Acurácia por fold: ", score)

ypred=model_selection.cross_val_predict(gnb,X,y,cv=5)

# Precisão
prec=precision_score(y, ypred, average='weighted')
print("Precision:", prec)

# Revocação (Recall)
recall=recall_score(y, ypred, average='weighted')
print("Revocação:", recall)

# f1 score
f1=f1_score(y, ypred, average='macro')
print("f1 score:", f1)

# Matriz de confusão
cm=confusion_matrix(y, ypred)
print(cm)

Melhores parâmetros encontrados:  {'priors': [0.2, 0.8], 'var_smoothing': 1e-08}
Acurácia média:  0.8158986175115206
Desvio padrão:  0.010130325426114278
Acurácia por fold:  [0.81663837 0.79796265 0.81494058 0.82852292 0.82142857]
Precision: 0.8577161341689404
Revocação: 0.8158967391304348
f1 score: 0.8152513760788064
[[1288  496]
 [  46 1114]]


###Logistic Regression(LR)
___
Logistic Regression é um algoritmo de aprendizado de máquina utilizado em problemas de classificação. Ele é utilizado para prever a probabilidade de uma determinada instância pertencer a uma classe específica, com base em variáveis de entrada (características).

A ideia por trás do modelo de regressão logística é usar uma função logística para modelar a relação entre as variáveis de entrada e a saída, que é uma variável binária (0 ou 1, por exemplo). Essa função logística transforma a soma ponderada das variáveis de entrada em uma probabilidade de saída.

Durante o treinamento, o algoritmo ajusta os pesos das variáveis de entrada para minimizar a diferença entre as previsões do modelo e as saídas reais do conjunto de treinamento. Isso é feito usando uma função de custo, como a função de perda logística.

Uma vez treinado, o modelo pode ser usado para fazer previsões em novos dados, onde as características de entrada são conhecidas, mas as saídas são desconhecidas. O modelo calcula a probabilidade de cada possível classe de saída e, em seguida, prevê a classe com a probabilidade mais alta.

O Logistic Regression é um modelo simples e eficaz que pode ser facilmente interpretado e implementado em problemas de classificação binária ou multiclasse. Ele é amplamente utilizado em aplicações como análise de risco de crédito, previsão de mortalidade hospitalar e diagnóstico médico.

In [6]:
# Logistic Regression (LR)
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Definindo os valores dos hiperparâmetros para teste
valores_C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
valores_max_iter = [10, 50, 100, 500, 1000]

# Criando o dicionário com os hiperparâmetros
hiperparametros = dict(C=valores_C, max_iter=valores_max_iter)

clf = LogisticRegression()

gs=GridSearchCV(clf, hiperparametros, cv=5, n_jobs=-1)
gs.fit(X_val, y_val)

print("Melhores parâmetros encontrados: ", gs.best_params_)

clf = gs.best_estimator_
score = model_selection.cross_val_score(gnb, X, y, cv=5)
print("Acurácia média: ", score.mean())
print("Desvio padrão: ", score.std())
print("Acurácia por fold: ", score)

ypred=model_selection.cross_val_predict(clf,X,y,cv=5)

# Precisão
prec=precision_score(y, ypred, average='weighted')
print("Precision:", prec)

# Revocação (Recall)
recall=recall_score(y, ypred, average='weighted')
print("Revocação:", recall)

# f1 score
f1=f1_score(y, ypred, average='macro')
print("f1 score:", f1)

# Matriz de confusão
cm=confusion_matrix(y, ypred)
print(cm)
"""

'\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)\n\n# Definindo os valores dos hiperparâmetros para teste\nvalores_C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]\nvalores_max_iter = [10, 50, 100, 500, 1000]\n\n# Criando o dicionário com os hiperparâmetros\nhiperparametros = dict(C=valores_C, max_iter=valores_max_iter)\n\nclf = LogisticRegression()\n\ngs=GridSearchCV(clf, hiperparametros, cv=5, n_jobs=-1)\ngs.fit(X_val, y_val)\n\nprint("Melhores parâmetros encontrados: ", gs.best_params_)\n\nclf = gs.best_estimator_\nscore = model_selection.cross_val_score(gnb, X, y, cv=5)\nprint("Acurácia média: ", score.mean())\nprint("Desvio padrão: ", score.std())\nprint("Acurácia por fold: ", score)\n\nypred=model_selection.cross_val_predict(clf,X,y,cv=5)\n\n# Precisão\nprec=precision_score(y, ypred, average=\'weighted\')\nprint("Precision:", prec)\n\n# Revocação (Recall)\nrecall=recall_score(y, ypred, average=\'weighted\')\nprint("Revocação:", recall)\n\n# f1 score\nf1=f1_scor

##Regressores
Um regressor, por outro lado, é um modelo de machine learning usado para prever valores numéricos. Ele é usado para prever um resultado contínuo, como o preço de uma casa, com base em um conjunto de variáveis independentes.

###KNN REGRESSOR
___
KNN Regressor é um algoritmo de aprendizado de máquina que se baseia em encontrar os "vizinhos" mais próximos de uma determinada amostra para determinar sua classe. O número de vizinhos considerados é definido por um parâmetro chamado K. Para o KNN Regressor, o objetivo é prever um valor numérico em vez de uma classe. Para isso, os valores numéricos dos vizinhos mais próximos são utilizados para calcular a média ou a mediana, que é então utilizada como a predição para a amostra em questão.

O KNN Regressor é uma variação do KNN que é usado especificamente para problemas de regressão. Em vez de prever uma classe para uma amostra de entrada, ele estima um valor numérico com base nas saídas dos k vizinhos mais próximos. Por exemplo, se estivermos tentando prever o preço de uma casa com base em seus recursos, podemos usar o KNN Regressor para encontrar os k vizinhos mais próximos da casa em questão e estimar o preço com base nos preços desses vizinhos.

Em resumo, a principal diferença entre o KNN e o KNN Regressor é que o KNN é usado para problemas de classificação e regressão, enquanto o KNN Regressor é usado apenas para problemas de regressão.

In [11]:
# KNN Regressor
folds=5

# separando uma parte para base de validação (20%)
X, X_val, y, y_val = train_test_split(X, y, train_size=0.8, random_state=42)

model = KNeighborsRegressor()
parameters = [{
      'n_neighbors': [1, 3, 5, 7, 9, 11],
      'weights': ['uniform', 'distance'],
      'p':[1,2]
    }]

# GridSearch para customizar os parâmetros sobre base de validação
gs = GridSearchCV(model, parameters, scoring = 'r2', cv=folds, n_jobs=-1)
gs.fit(X_val, y_val)

# from tabulate import tabulate
# import pandas as pd
# df=gs.cv_results_
# print(tabulate(df, headers='keys', tablefmt='psql'))
print("Melhores parâmetros encontrados: ", gs.best_params_)

# Recuperando os melhores parâmetros
model=gs.best_estimator_

# Usando a validação cruzada com 5 folds neste exemplo.
T=5 # número de pastas ou folds
result = model_selection.cross_val_score(model, X, y, cv=T)

# Mostrando R2 médio e desvio padrão calculados na validação cruzada.
print("\nCross Validation Results %d folds:" % T)
print("Acurácia R2 médio: %.5f" % result.mean())
print("Desvio padrão: %.5f" % result.std())

# Calculando o valor para cada exemplo de teste
y_pred = model_selection.cross_val_predict(model, X, y, cv=T)

# Exemplo mostrando o resultado previsto para a primeira instância de teste
print("Primeira instância na base de teste apresenta valor diabetes: %d" % y_pred[0])

# Calculando o erro médio absoluto
mae=mean_absolute_error(y, y_pred)
print("Mean Absolute Error (MAE) calculado na base de teste: %.5f" % mae)

Melhores parâmetros encontrados:  {'n_neighbors': 7, 'p': 1, 'weights': 'distance'}

Cross Validation Results 5 folds:
Acurácia R2 médio: 0.49743
Desvio padrão: 0.03079
Primeira instância na base de teste apresenta valor diabetes: 0
Mean Absolute Error (MAE) calculado na base de teste: 0.22395


###Árvore de Decisão Regressor
___
A árvore de decisão regressor é uma variação da árvore de decisão que é utilizada em problemas de regressão. Ao invés de prever classes, como na árvore de decisão para classificação, a árvore de decisão regressor é usada para prever valores contínuos.

O funcionamento é semelhante à árvore de decisão para classificação. A árvore é construída a partir de um conjunto de dados de treinamento, onde cada nó interno é dividido com base em um atributo que otimiza a divisão dos dados em subconjuntos mais homogêneos em termos de valor da variável resposta. Em cada nó folha, um valor médio da variável resposta é atribuído, representando a previsão para novos exemplos.

A construção da árvore de decisão regressor é realizada utilizando medidas de impureza, como a soma dos erros quadrados (SSE) ou o desvio médio absoluto (MAD). O objetivo é minimizar a soma dos erros quadrados ou o desvio médio absoluto, para que a árvore possa fazer previsões mais precisas.

Após a construção da árvore, o algoritmo pode ser aplicado para fazer previsões em novos exemplos, percorrendo a árvore a partir do nó raiz e seguindo os caminhos de acordo com os valores dos atributos.

Uma das vantagens da árvore de decisão regressor é a interpretabilidade do modelo, uma vez que as decisões são tomadas a partir de regras simples que podem ser visualizadas e entendidas facilmente. No entanto, assim como em outros modelos de árvore de decisão, há o risco de overfitting, onde a árvore se ajusta demais aos dados de treinamento e perde capacidade de generalização para novos dados.

In [13]:
# Exemplo de Decision Tree Regression

parameters = [{'max_depth': [3, 5, 10, 20],
               'min_samples_split': [3, 5, 10],
               'criterion':['absolute_error', 'squared_error'],
               'splitter':['best', 'random']},]

folds = 5
model = DecisionTreeRegressor()

# separando uma parte para base de validação (20%)
X, X_val, y, y_val = train_test_split(X, y, train_size = 0.8, random_state = 42)

# GridSearch para customizar os parâmetros sobre base de validação
gs = GridSearchCV(model, parameters, scoring = 'r2', cv = folds, n_jobs = 5)
gs.fit(X_val, y_val)

#df = gs.cv_results_
# print(tabulate(df, headers = 'keys', tablefmt = 'psql'))
print("Melhores parâmetros encontrados: ", gs.best_params_)

# Definindo a técnica a ser utilizada
model = gs.best_estimator_

# Usando a validação cruzada com 5 folds neste exemplo.
T = 5 # número de pastas ou folds
result = model_selection.cross_val_score(model, X, y, cv = T)

# Mostrando R2 médio e desvio padrão calculados na validação cruzada.
print("\nCross Validation Results %d folds:" % T)
print("R2 médio: %.5f" % result.mean())
print("Mean Std: %.5f" % result.std())
print("Acurácia por fold: ", result)

# Calculando o valor para cada exemplo de teste
y_pred = model_selection.cross_val_predict(model, X, y, cv = T)

# Exemplo mostrando o resultado previsto para a primeira instância de teste
print("Primeira instância na base de teste apresenta valor: %d" % y_pred[0])

# Calculando o erro médio absoluto
mae = mean_absolute_error(y, y_pred)
print("Mean Absolute Error (MAE) calculado na base de teste: %.5f" % mae)

# Calculando o erro médio absoluto
mse = mean_squared_error(y, y_pred)
print("Mean Squared Error (MSE) calculado na base de teste: %.5f" % mse)

Melhores parâmetros encontrados:  {'criterion': 'squared_error', 'max_depth': 10, 'min_samples_split': 5, 'splitter': 'random'}

Cross Validation Results 5 folds:
R2 médio: 0.60083
Mean Std: 0.02127
Acurácia por fold:  [0.60834516 0.58786758 0.5678955  0.6096485  0.63040539]
Primeira instância na base de teste apresenta valor: 1
Mean Absolute Error (MAE) calculado na base de teste: 0.16491
Mean Squared Error (MSE) calculado na base de teste: 0.09802


##Conclusão
___
Os classificadores e regressores são técnicas fundamentais em Machine Learning, que visam construir modelos capazes de prever a classe ou valor de saída de uma nova instância com base em um conjunto de dados de treinamento. Ambos os tipos de modelos são construídos utilizando um algoritmo de aprendizado supervisionado, onde o modelo é treinado com um conjunto de dados rotulados para aprender a mapear as entradas para saídas corretas.

Os classificadores são modelos que tentam prever a classe de uma nova instância, atribuindo uma ou mais categorias para ela. O objetivo é criar um modelo que seja capaz de classificar novas instâncias com alta precisão. Alguns exemplos de classificadores incluem Árvores de Decisão, Naive Bayes, K-Nearest Neighbors, Random Forest e Support Vector Machines. Cada um desses algoritmos tem suas próprias vantagens e desvantagens, sendo a escolha do melhor modelo dependente das características do conjunto de dados e do problema em questão.

Já os regressores são modelos que tentam prever um valor numérico de saída para uma nova instância. O objetivo é criar um modelo que seja capaz de prever valores precisos para novas instâncias. Alguns exemplos de regressores incluem Regressão Linear, Regressão Logística, Árvores de Regressão e Support Vector Regression. Assim como os classificadores, cada algoritmo tem suas próprias vantagens e desvantagens, sendo a escolha do melhor modelo dependente das características do conjunto de dados e do problema em questão.

Em resumo, tanto os classificadores quanto os regressores são ferramentas poderosas em Machine Learning para previsão de resultados. A escolha entre um modelo classificador ou regressor vai depender do tipo de variável de saída que se deseja prever: se é uma variável categórica ou discreta, um classificador é o mais adequado; se é uma variável contínua, um regressor é mais apropriado.